In [1]:
import pandas as pd

data = pd.read_csv("CO2_passenger_cars_v12-utf8.csv", encoding = 'utf8')

/Users/kaio/Documents/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (15,16,17,18,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
import numpy as np

diesel_col = data[data["Ft"] == "DIESEL"]["e (g/km)"]